In [7]:
import pandas as pd
from keras.utils import to_categorical
import numpy as np

labels = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/activity_labels.txt", header=None, sep="\s+")

X_train = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt", header=None, sep="\s+")
y_train = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt", header=None)
X_test_val = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/test/X_test.txt", header=None, sep="\s+")
y_test_val = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt", header=None)

X_test = X_test_val[0:round(0.9*X_test_val.shape[0])] 
X_val = X_test_val[round(0.9*X_test_val.shape[0]):] 
y_test = y_test_val[0:round(0.9*y_test_val.shape[0])] 
y_val = y_test_val[round(0.9*y_test_val.shape[0]):] 

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
y_train = to_categorical(y_train)
y_test_cat = to_categorical(y_test)
y_val = to_categorical(y_val)

X_train.shape, y_train.shape, X_test.shape, y_test.shape, labels.shape

((7352, 561, 1), (7352, 7), (2652, 561), (2652, 1), (6, 2))

In [2]:
from keras import Sequential
from keras.layers import Bidirectional, LSTM, Dense, Dropout
import keras

modelo = Sequential()
modelo.add(Bidirectional(LSTM(64, return_sequences=True)))
modelo.add(Dropout(0.5))
modelo.add(Bidirectional(LSTM(64)))
modelo.add(Dropout(0.5))
modelo.add(Dense(7, activation="sigmoid"))

modelo.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy", "Precision", "Recall"])

In [3]:
historico = modelo.fit(X_train, y_train, epochs=20, batch_size=10, validation_data=(X_val, y_val))
modelo.save("ModeloBiLSTM.keras")

Epoch 1/20


736/736 [==============================] - 182s 239ms/step - loss: 0.2994 - accuracy: 0.4382 - precision: 0.5127 - recall: 0.2218 - f1_score: 0.3768 - val_loss: 0.1949 - val_accuracy: 0.6949 - val_precision: 0.6920 - val_recall: 0.5254 - val_f1_score: 0.6070
Epoch 2/20
736/736 [==============================] - 179s 243ms/step - loss: 0.1850 - accuracy: 0.7174 - precision: 0.7334 - recall: 0.6336 - f1_score: 0.6075 - val_loss: 0.1112 - val_accuracy: 0.8339 - val_precision: 0.8495 - val_recall: 0.8034 - val_f1_score: 0.7230
Epoch 3/20
736/736 [==============================] - 177s 241ms/step - loss: 0.1354 - accuracy: 0.8152 - precision: 0.8199 - recall: 0.7773 - f1_score: 0.6935 - val_loss: 0.0573 - val_accuracy: 0.9797 - val_precision: 0.9891 - val_recall: 0.9254 - val_f1_score: 0.8419
Epoch 4/20
736/736 [==============================] - 182s 247ms/step - loss: 0.1165 - accuracy: 0.8452 - precision: 0.8445 - recall: 0.8192 - f1_score: 0.7199 - val_loss: 0.0456 - val_acc

In [8]:
_, precisao, precision, recall = modelo.evaluate(X_test, y_test_cat)
f1_score = (2 * precision * recall) / (precision + recall)
print(f"Precisão do modelo: {precisao*100} %")
print(f"Precision do modelo: {precision*100} %")
print(f"Recall do modelo: {recall*100} %")
print(f"F1-Score do modelo: {f1_score*100} %")

83/83 [==============================] - 9s 107ms/step - loss: 0.1248 - accuracy: 0.8314 - precision: 0.8299 - recall: 0.8318 - f1_score: 0.7071
Precisão do modelo: 83.1447958946228 %
Precision do modelo: 82.99473524093628 %
Recall do modelo: 83.18250179290771 %
F1-Score do modelo: 98.07497262954712 %


In [ ]:
import matplotlib.pyplot as plt
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

plt.plot(historico.history["accuracy"])
plt.plot(historico.history["val_accuracy"])
plt.grid()
plt.title("Evolução da precisão/validação do Treino")
plt.legend(["Precisão", "Validação"])
plt.xlabel("Época")
plt.xlabel("Precisão")
plt.savefig("Evolução Precisão Validação BiLSTM", bbox_inches="tight")

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_previsto = modelo.predict(X_test)
y_previsto = [np.argmax(y) for y in y_previsto]

cm = confusion_matrix(y_test, y_previsto)
display = ConfusionMatrixDisplay(cm, display_labels=labels[1])
display.plot(xticks_rotation="vertical")
display.figure_.savefig("Matriz de Confusão BiLSTM", bbox_inches="tight")